# Exemplos de tools funcionais

Agora vamos abordar a criação de duas tools com utilidade real para você entender como seria todo o processo.

## Criando um tool de busca de temperatura

https://open-meteo.com/en/docs

In [40]:
# === IMPORTAÇÕES NECESSÁRIAS ===
import requests
import datetime
import wikipedia
from langchain.agents import tool
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.agents import AgentFinish

# === TOOL: Temperatura ===
class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''
    URL = 'https://api.open-meteo.com/v1/forecast'
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f"A temperatura atual é {temp_atual}°C"
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')


# === TOOL: Wikipedia ===
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)


# === DEFININDO TOOLS ===
tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}


# === MODELO ===
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")  # ou gemini-1.5-pro, gemini-1.5-flash

# === PROMPT ===
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Pense com cuidado ao categorizar o texto conforme as instruções'),
    ('user', '{input}')
])


# === AGENTE COMBINADO COM TOOLS ===
chain = (
    prompt
    | model.bind(functions=tools_json)
    | OpenAIFunctionsAgentOutputParser()
)

# === GERENCIADOR DE EXECUÇÃO ===
def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].invoke(resultado.tool_input)


# === CADEIA FINAL ===
final_chain = chain | roteamento


# === EXEMPLO DE USO ===
resposta = final_chain.invoke({'input': 'Qual a temperatura agora em São Paulo na latitude -23.55 e longitude -46.63?'})
print(resposta)

resposta2 = final_chain.invoke({'input': 'Me diga algo sobre futebol americano'})
print(resposta2)


A temperatura atual é 19.0°C
Título da página: Futebol americano
Resumo: O futebol americano (em inglês American football, referido simplesmente como futebol nos Estados Unidos e Canadá), também referido às vezes como gridiron football, é um esporte de equipe jogado por dois times de onze jogadores em um campo retangular com traves em cada extremidade. A equipe de ataque, com a posse da bola oval, tenta avançar pelo campo correndo com a bola ou lançando-a, enquanto a defesa, que não possui a bola, tenta impedir o avanço do ataque e tomar o controle da bola para si. O futebol americano é um jogo caracterizado pela conquista de território. O ataque deve avançar pelo menos dez jardas em quatro tentativas ou jogadas (chamadas de descidas ou downs); se falharem, entregam a bola para a defesa, mas se conseguirem avançar essas dez jardas, recebem um novo conjunto de quatro tentativas para continuar avançando (a campanha é chamada de drive). O jogo é vencido pela equipe com o maior número de p